# Lab-10-3 Visdom

## 지난시간까지
- MNIST-CNN

## 오늘은
- Visdom사용법을 익히고 MNIST-CNN 학습에 적용해보자.

## Visdom 설치
`pip install visdom`

## Visdom 서버 켜기
`python -m visdom.server`

이러면 `http://localhost:8097`에 열려 있다는 메시지 출력됨

In [ ]:
pip install visdom

## visdom 사용법으로 이번에 배울 것
- Text
- image
- images
- Line plot

## 지금부터
- visdom을 사용법을 익히고 MNIST-CNN loss graph까지 적용

### 10-3-1 Visdom Example

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

### import visdom

In [2]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


### Text

In [3]:
vis.text("Hello, world!", env="main")
# env="main"은 있어도 되고 없어도 되는데, 나중에 한번에 모든 창 끌 때 사용할 수 있음.

'window_3b51072d11ab70'

### Image

In [4]:
a = torch.randn(3, 200, 200)  # 3으로 하면 rgb로 잡히고, 200x200
vis.image(a)

'window_3b51072e167b16'

### Images

In [5]:
vis.images(torch.Tensor(3, 3, 28, 28))
# image 3개 띄워진 것 볼 수 있음. 28x28 이미지 3개

'window_3b51072ef3d308'

### example (using MNIST and CIFAR10)

In [6]:
MNIST = dsets.MNIST(root="./MNIST_data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10", train=True, transform=torchvision.transforms.ToTensor(), download=True)

Files already downloaded and verified


#### CIFAR10

In [7]:
data = cifar10.__getitem__(0)  # cifar10의 첫번째 데이터 가져옴
print(data[0].shape)           # [3, 32, 32] : 3채널(rgb) 32x32 그림
vis.images(data[0], env="main")

torch.Size([3, 32, 32])


'window_3b510730bfe896'

#### MNIST

In [8]:
data = MNIST.__getitem__(0)
print(data[0].shape)          # [1, 28, 28] : 1채널(gray) 28x28 그림
vis.images(data[0], env="main")

torch.Size([1, 28, 28])


'window_3b51073170fbec'

#### Check dataset
데이터 한 번에 보기

In [9]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST,
                                         batch_size=32,
                                         shuffle=False)

In [10]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)   # [32, 1, 28, 28] : 1채널 28x28 이미지 32개
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [11]:
vis.close(env="main")   # main인 애들 다 꺼줘

''

#### Line Plot

학습을 진행하면서 loss가 어떻게 되고 있는지를 그래프로 보고 싶을 수 있음. 그래프로 표현하는 방법

In [12]:
Y_data = torch.randn(5)
# 원소 5개짜리 리스트. X 값을 넣지 않으면 x축 값은 무조건 0부터 1까지의 값으로만 선언이 된다.
plt = vis.line(Y=Y_data)

In [13]:
X_data = torch.Tensor([1, 2, 3, 4, 5])
# x 값을 넣어주면 x축 값이 1, 2, 3, 4, 5 이런 식으로 그대로 들어감.
plt = vis.line(Y=Y_data, X=X_data)

#### Line update

In [14]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update="append")

'window_3b510735ac3258'

#### multiple Line on single windows

In [15]:
num = torch.Tensor(list(range(0, 10)))
print(num.shape)
num = num.view(-1, 1)
print(num.shape)
num = torch.cat((num, num), dim=1)
print(num.shape)

plt = vis.line(Y=torch.randn(10, 2), X=num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


#### Line info

In [16]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title="Test", showlegend=True))

In [17]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title="Test", legend=["1번"], showlegend=True))

In [18]:
plt = vis.line(Y=torch.randn(10, 2), X=num, opts=dict(title="Test", legend=["1번", "2번"]))

#### make function for update line

In [19]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update="append"
            )

In [20]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) * i
    loss_tracker(plt, loss, torch.Tensor([i]))

#### close the window

In [21]:
vis.close(env="main")

''

이 외에도 엄청 많은 기능들이 있다!

### 10-3 MNIST-CNN with Visdom

10-2 할 때랑 똑같은 코드인데 visdom만 넣은 것

In [23]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

#### import visdom

In [24]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

#### define loss_tracker

In [25]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update="append"
            )

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [27]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [28]:
#MNIST dataset

mnist_train = dsets.MNIST(root="MNIST_data/",
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root="MNIST_data/",
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [29]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [30]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [31]:
model = CNN().to(device)

value = (torch.Tensor(1, 1, 28, 28)).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


In [32]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### make plot

In [33]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

#### train with loss_tracker

In [34]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoth:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

[Epoth:1] cost = 0.1293681412935257
[Epoth:2] cost = 0.04197239503264427
[Epoth:3] cost = 0.029367033392190933
[Epoth:4] cost = 0.02299959398806095
[Epoth:5] cost = 0.01744411513209343
[Epoth:6] cost = 0.014136048033833504
[Epoth:7] cost = 0.01292012445628643
[Epoth:8] cost = 0.011273977346718311
[Epoth:9] cost = 0.009665490128099918
[Epoth:10] cost = 0.007492149714380503
[Epoth:11] cost = 0.009115600027143955
[Epoth:12] cost = 0.0069937133230268955
[Epoth:13] cost = 0.008123529143631458
[Epoth:14] cost = 0.006831460632383823
[Epoth:15] cost = 0.007010275032371283
Learning Finished!


![snapshot](./10-3-0.png)

In [35]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/Users/sanghoon/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/sanghoon/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9733999967575073


### 오늘 같이 해본 것들은?
- Visdom 사용법
- 실제 학습에 적용해보기

### What's Next?
- 내가 가진 사진을 이용해서 학습을 해볼 수 없을까?